
# Bibliotecas Necessárias para o Jupyter Notebook

Para rodar este Jupyter Notebook com sucesso, você precisa instalar as seguintes bibliotecas:

1. transformers
2. scikit-learn
3. torch (PyTorch)
4. accelerate

## Instalação

Você pode instalar todas essas bibliotecas com um único comando pip:


In [6]:

#Descomente para instalar as bibliotecas pela primeira vez
#!pip install transformers[torch] scikit-learn accelerate


Este comando fará o seguinte:

- Instala a biblioteca `transformers` com suporte para PyTorch
- Instala o PyTorch (`torch`)
- Instala o `scikit-learn` para funções de divisão de dados
- Instala o `accelerate`, necessário para usar o `Trainer` com PyTorch

## Importante

Após executar este comando de instalação, você deve reiniciar o kernel do Jupyter Notebook para garantir que todas as bibliotecas sejam carregadas corretamente.

## Funcionalidades Habilitadas

Com estas bibliotecas instaladas, você será capaz de:

- Carregar e usar modelos e tokenizadores do DistilBERT
- Realizar a divisão de dados de treino e teste
- Criar e treinar o modelo usando o `Trainer` do Hugging Face
- Avaliar o modelo treinado

## Nota

Se você estiver usando um ambiente virtual ou conda, certifique-se de ativá-lo antes de instalar estas bibliotecas e executar o notebook.

In [ ]:
#!pip install transformers[torch] scikit-learn accelerate


Este comando fará o seguinte:

- Instala a biblioteca `transformers` com suporte para PyTorch
- Instala o PyTorch (`torch`)
- Instala o `scikit-learn` para funções de divisão de dados
- Instala o `accelerate`, necessário para usar o `Trainer` com PyTorch

## Importante

Após executar este comando de instalação, você deve reiniciar o kernel do Jupyter Notebook para garantir que todas as bibliotecas sejam carregadas corretamente.

## Funcionalidades Habilitadas

Com estas bibliotecas instaladas, você será capaz de:

- Carregar e usar modelos e tokenizadores do DistilBERT
- Realizar a divisão de dados de treino e teste
- Criar e treinar o modelo usando o `Trainer` do Hugging Face
- Avaliar o modelo treinado

## Nota

Se você estiver usando um ambiente virtual ou conda (como estamos usanbdo nessa aula), certifique-se de ativá-lo antes de instalar estas bibliotecas e executar o notebook.

In [7]:
import torch
print(torch.__version__)

2.4.1+cu121


In [3]:
# Importar bibliotecas necessárias
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

# Dados de exemplo
textos = [
    "Eu adorei este filme!",
    "O produto é péssimo, não recomendo.",
    "Excelente qualidade e ótimo atendimento.",
    "Não gostei, esperava mais.",
    "Maravilhoso, superou minhas expectativas!"
]
labels = [1, 0, 1, 0, 1]  # 1 = Positivo, 0 = Negativo

# Dividir os dados em treino e validação
train_texts, val_texts, train_labels, val_labels = train_test_split(textos, labels, test_size=0.2)

# Carregar tokenizer e modelo pré-treinado
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Tokenização
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Criar Dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# Configurações de treinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
)

# Criar trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo
trainer.evaluate()


/home/anderson/miniconda3/envs/scoras_academy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/anderson/miniconda3/envs/scoras_academy/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                             
100%|██████████| 4/4 [00:00<00:00,  7.93it/s]

{'eval_loss': 0.6393745541572571, 'eval_runtime': 0.0532, 'eval_samples_per_second': 18.808, 'eval_steps_per_second': 18.808, 'epoch': 1.0}


                                             
100%|██████████| 4/4 [00:01<00:00,  2.35it/s]


{'eval_loss': 0.6681358218193054, 'eval_runtime': 0.0064, 'eval_samples_per_second': 155.685, 'eval_steps_per_second': 155.685, 'epoch': 2.0}
{'train_runtime': 1.7034, 'train_samples_per_second': 4.697, 'train_steps_per_second': 2.348, 'train_loss': 0.7418535351753235, 'epoch': 2.0}


100%|██████████| 1/1 [00:00<00:00, 2396.75it/s]


{'eval_loss': 0.6681358218193054,
 'eval_runtime': 0.0066,
 'eval_samples_per_second': 151.244,
 'eval_steps_per_second': 151.244,
 'epoch': 2.0}

# Análise dos Resultados do Treinamento

Após o treinamento do modelo, obtivemos os seguintes resultados:

1. **eval_loss: 0.6681358218193054**
   - Este é o valor da função de perda (loss) na avaliação final do modelo.
   - A perda indica o quão bem o modelo está performando. Quanto menor, melhor.
   - Um valor de 0.668 é moderado, sugerindo que o modelo aprendeu algo, mas ainda há espaço para melhorias.

2. **eval_runtime: 0.0066**
   - Este é o tempo total, em segundos, que levou para avaliar o modelo.
   - Um tempo de 0.0066 segundos é muito rápido, indicando que a avaliação foi feita em uma pequena quantidade de dados.

3. **eval_samples_per_second: 151.244**
   - Indica quantas amostras o modelo pode processar por segundo durante a avaliação.
   - Um valor de 151.244 significa que o modelo é capaz de avaliar aproximadamente 151 amostras por segundo.

4. **eval_steps_per_second: 151.244**
   - Similar ao anterior, mas mede os passos de avaliação por segundo.
   - Neste caso, é igual ao número de amostras por segundo, sugerindo que cada passo processa uma amostra.

5. **epoch: 2.0**
   - Indica que o modelo completou 2 épocas de treinamento.
   - Uma época é uma passagem completa por todos os dados de treinamento.

## Interpretação Geral

- O modelo foi treinado por 2 épocas completas.
- A avaliação foi rápida (0.0066 segundos), sugerindo um conjunto de dados pequeno.
- A perda de 0.668 indica que o modelo aprendeu padrões nos dados, mas ainda há margem para melhorias.
- O modelo é capaz de processar cerca de 151 amostras por segundo durante a avaliação, o que é uma velocidade razoável.

Para melhorar o desempenho, você poderia considerar:
1. Aumentar o número de épocas de treinamento.
2. Usar um conjunto de dados maior.
3. Ajustar os hiperparâmetros do modelo.
4. Experimentar técnicas de regularização para reduzir o overfitting, se necessário.

# 5 Tarefas Potenciais para o Modelo DistilBERT Treinado

1. **Análise de Sentimentos em Redes Sociais**
   - Classificar tweets, posts do Facebook ou comentários do Instagram como positivos, negativos ou neutros.
   - Útil para empresas monitorarem a percepção da marca online.

2. **Classificação de Feedback de Clientes**
   - Categorizar automaticamente avaliações de produtos ou serviços.
   - Ajuda empresas a identificar rapidamente áreas de satisfação ou insatisfação dos clientes.

3. **Triagem de E-mails**
   - Classificar e-mails como spam, importantes, promocionais, etc.
   - Pode melhorar a eficiência na gestão de caixas de entrada corporativas.

4. **Análise de Títulos de Notícias**
   - Categorizar manchetes de notícias por tópico (política, esportes, tecnologia, etc.).
   - Útil para agregadores de notícias ou para análise de tendências na mídia.

5. **Detecção de Intenção em Chatbots**
   - Identificar a intenção do usuário em consultas de chatbot (por exemplo, fazer uma pergunta, solicitar suporte, fazer uma reclamação).
   - Melhora a resposta automatizada em sistemas de atendimento ao cliente.